第三方工具是在别的数据集上面训练出来的，未必适合你的应用场景。

例如有些情感分析工具更适合分析新闻，有的更善于处理微博数据……你拿过来，却是要对店铺评论信息做分析。

### 数据

+ 数据是大众点评网上的餐厅评论数据，选取了评论文本和评星（1-5星），从这些数据中随机筛选评星为1,2,4,5的，隔各500条评论数据，共2k条，之所以没有选择评星为3，是因为：
+ 只希望对情感做出（正和负）二元分类，4和5可以看做正向情感，1和2是负向，无法划定3
+ 所以，为了避免边界不清晰造成的混淆，只好把标为3的内容丢掉

### 模型选择

使用机器学习的时候，会遇到模型选择的问题，scikit-learn不仅提供了很多借口，还给了我们一个非常贴心的小抄（cheat-sheeet）
+ 下面这幅图，看似很混乱，密密麻麻，但是一个很好的迷宫指南
    + 很明显，分了四大块，从左上开始，顺时针，分别是： 分类，回归，降维，聚类四种
    + 其中绿色的方框，是各种机器学习模型，蓝色的圆圈，是做判断的地方
+ 我们目前要处理的是分类问题，所以在第一个粉色区域里找（可以右击在新标签页中打开，然后放大看）
    + 从start开始， >50 sample，Yes
    + predicting a category 预测类别，Yes
    + labeled data 有标记数据 Yes
    + <100k 数据存储空间<100k Yes
    + Linear SVC 如果不工作，再继续
    + Text Data Yes
    + Naive Bayes 朴素贝叶斯

![avatar](https://upload-images.jianshu.io/upload_images/64542-1889ba0aa5e9c3a8..png?imageMogr2/auto-orient/)

**scikit-learn是个很好的库，如果需要使用经典机器学习的模型（可以理解为深度学习之外的所有模型），推荐先尝试scikit-learn**

词袋模型的一张很形象的图

![avatar](https://ldabook.com/Images/BagOfWords.png)

词袋模型不考虑词的出现顺序，也不考虑词语和前后词语之间的连接。每个词都被当作一个独立的特征来看待
+ 缺点：这样不精确，没有考虑顺序和上下文联系。
    + 虽然对文本的顺序，结构考虑的越周全，模型可以获得的信息就越多，但是凡事都有成本
    + 比较 独立考虑单词 和 考虑连续n个词语（n-gram） 造成的模型维度差异
    
所以为了简单期间，还是先用词袋模型就好


1. 你可能担心计算机处理起中文的词语，跟处理英文词语有所不同，这种担心没必要
2. 因为其实计算机连英语单词也看不懂，在它眼里，不管是什么自然语言的词汇，都只是特定组合的字符而已
3. 不管是处理什么语言，都需要处理一种词，叫停用词：
 
中文维基百科的定义：
> 在信息检索中，为节省存储空间，提高搜索效率，在处理自然语言数据（或文本）之前或之后会自动过滤掉某些字或词，这些字或词即被称为Stop Words（停用词）      

所以现在我们不做索引，做分类，那些我们不打算用来做特征词的单词，就可以当成停用词

### 代码实现

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('data.csv',encoding='gb18030')

注意，为了让中文文本csv文件能顺序正确不乱码的在excel中打开，编码设置成了gb18030，或者所谓的utf-8 含 BOM

In [3]:
df.head()

,comment,star
0,口味：不知道是我口高了，还是这家真不怎么样。 我感觉口味确实很一般很一般。上菜相当快，我敢说...,2
1,菜品丰富质量好，服务也不错！很喜欢！,4
2,说真的，不晓得有人排队的理由，香精香精香精香精，拜拜！,2
3,菜量实惠，上菜还算比较快，疙瘩汤喝出了秋日的暖意，烧茄子吃出了大阪烧的味道，想吃土豆片也是口...,5
4,先说我算是娜娜家风荷园开业就一直在这里吃 每次出去回来总想吃一回 有时觉得外面的西式简餐总是...,4


数据读入正确

In [4]:
df.shape

(2000, 2)

数据读取完整，2000行，每行两个字段

### 标记情感值

+ 我们并不打算分四个类别，只打算进行二元分类，分成正向和负向，所以需要一个函数，将评星转为情感
+ 评星>3 正向情感 取值为1， <3 负向情感，取值为0


In [3]:
def make_label(df):
    df['sentiment']=df['star'].apply(lambda x:1 if x>3 else 0)

In [4]:
make_label(df)

In [7]:
df.head()

,comment,star,sentiment
0,口味：不知道是我口高了，还是这家真不怎么样。 我感觉口味确实很一般很一般。上菜相当快，我敢说...,2,0
1,菜品丰富质量好，服务也不错！很喜欢！,4,1
2,说真的，不晓得有人排队的理由，香精香精香精香精，拜拜！,2,0
3,菜量实惠，上菜还算比较快，疙瘩汤喝出了秋日的暖意，烧茄子吃出了大阪烧的味道，想吃土豆片也是口...,5,1
4,先说我算是娜娜家风荷园开业就一直在这里吃 每次出去回来总想吃一回 有时觉得外面的西式简餐总是...,4,1


起码从前五行来看，情感取值就是根据我们设定的规则，从评星数量转化而来

### 把特征和标签拆开（X,Y）

In [5]:
X=df[['comment']]
Y=df.sentiment

**!注意，这里就是两个中括号，这样之后，X会被认为是一个DataFrame，而Y只会被认为是一个Series（序列）**

In [9]:
type(X)

pandas.core.frame.DataFrame

In [10]:
type(Y)

pandas.core.series.Series

In [11]:
Y.shape

(2000,)

In [12]:
X.shape

(2000, 1)

也就是2000行，1列

In [13]:
X.head()

,comment
0,口味：不知道是我口高了，还是这家真不怎么样。 我感觉口味确实很一般很一般。上菜相当快，我敢说...
1,菜品丰富质量好，服务也不错！很喜欢！
2,说真的，不晓得有人排队的理由，香精香精香精香精，拜拜！
3,菜量实惠，上菜还算比较快，疙瘩汤喝出了秋日的暖意，烧茄子吃出了大阪烧的味道，想吃土豆片也是口...
4,先说我算是娜娜家风荷园开业就一直在这里吃 每次出去回来总想吃一回 有时觉得外面的西式简餐总是...


### 分词

In [6]:
import jieba

In [7]:
def chinese_cut_word(text):
    return " ".join(jieba.cut(text))

In [8]:
X['cutted_comment']=X.comment.apply(chinese_cut_word)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\shanshan\AppData\Local\Temp\jieba.cache
Loading model cost 1.198 seconds.
Prefix dict has been built succesfully.
D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [17]:
X.head()

,comment,cutted_comment
0,口味：不知道是我口高了，还是这家真不怎么样。 我感觉口味确实很一般很一般。上菜相当快，我敢说...,口味 ： 不 知道 是 我口 高 了 ， 还是 这家 真 不怎么样 。 我 感觉 口味 ...
1,菜品丰富质量好，服务也不错！很喜欢！,菜品 丰富 质量 好 ， 服务 也 不错 ！ 很 喜欢 ！
2,说真的，不晓得有人排队的理由，香精香精香精香精，拜拜！,说真的 ， 不 晓得 有人 排队 的 理由 ， 香精 香精 香精 香精 ， 拜拜 ！
3,菜量实惠，上菜还算比较快，疙瘩汤喝出了秋日的暖意，烧茄子吃出了大阪烧的味道，想吃土豆片也是口...,菜量 实惠 ， 上菜 还 算 比较 快 ， 疙瘩汤 喝出 了 秋日 的 暖意 ， 烧茄子 吃...
4,先说我算是娜娜家风荷园开业就一直在这里吃 每次出去回来总想吃一回 有时觉得外面的西式简餐总是...,先说 我 算是 娜娜 家风 荷园 开业 就 一直 在 这里 吃 每次 出去 回来 总想 ...


In [18]:
Y.head()

0    0
1    1
2    0
3    1
4    1
Name: sentiment, dtype: int64

数据处理已经好了，接下来就是常规分析步骤

### 常规机器训练步骤

#### 将数据分为训练集和测试集

In [9]:
from  sklearn.model_selection import train_test_split

In [10]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,random_state=1)

设定random_state取值，是为了保证在不同环境中，随机数取值一致，以便验证模型的实际效果

In [21]:
X_train.shape

(1500, 2)

也就是说，**在默认模式下，train_test_split函数对训练集和测试集的划分比例为3:1**
对其他几个数据集进行shape，验证猜想

In [22]:
X_test.shape

(500, 2)

In [23]:
Y_test.shape

(500,)

In [24]:
Y_train.shape

(1500,)

####  处理停用词（一般这步不是在分词的时候就做了的吗）

In [11]:
def get_custom_stopwords(stop_words_file):
    with open(stop_words_file,encoding='utf8') as f:
        stopwords=f.read()
    stopwords_list=stopwords.split('\n')
    #custom_stopwords_list=[i for i in stopwords_list] # 这句其实没什么用啊，注释掉也没影响到结果
    return stopwords_list

In [12]:
stop_words_file='stopwordsHIT.txt'

In [13]:
stopwords=get_custom_stopwords(stop_words_file)

In [28]:
type(stopwords)

list

In [29]:
stopwords[-10:]

['呃', '呗', '咚', '咦', '喏', '啐', '喔唷', '嗬', '嗯', '嗳']

很明显，这些都是语气助词，去掉不影响语句的实质含义

#### 将分词向量化

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

这里为了显示停用词的作用，创立两个 CountVectorizer()实例，vect_stop 使用了停用词，vect没使用停用词

In [15]:
vect=CountVectorizer()

**下面这两个是为了SentimentLDA做的测试**

In [19]:
for i,word in enumerate(vect.get_feature_names()):
    print(i,word)
    break

0 00


In [20]:
from nltk.corpus import sentiwordnet as swn
synsets = swn.senti_synsets('happy')
#print(synsets)  #这个synsets是个对象

In [22]:
import numpy as np
posScore = np.mean([s.pos_score() for s in synsets])
negScore = np.mean([s.neg_score() for s in synsets])
print(posScore,negScore)    

0.5625 nan


D:\anaconda\lib\site-packages\numpy\core\fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
D:\anaconda\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


**所以NLTK中的那个Sentiwordnet是一个词有正的概率也有负的概率。改改，直接使用Hownet就好**

**看看那几个分布产出来的到底是什么玩意**

In [25]:
np.random.dirichlet(0.3*np.ones(3))

array([0.0022972 , 0.34769852, 0.65000428])

In [26]:
np.ones(3)

array([1., 1., 1.])

In [28]:
0.3*np.ones(3)

array([0.3, 0.3, 0.3])

In [31]:
np.random.dirichlet([0.3,0.3,0.3])

array([1.26719183e-01, 1.11248449e-06, 8.73279704e-01])

**所以狄利克雷分布属于random，也是随机数的一种，反正就是有关系就好，狄利克雷分布和多项式分布是共轭分布，相乘之后的结果还是满足狄利克雷分布，知道这个就好了**

In [17]:
term_matrix=pd.DataFrame(vect.fit_transform(X_train.cutted_comment).toarray(),columns=vect.get_feature_names())

**穿插SentimentLDA测试内容**

In [16]:
matrix=vect.fit_transform(X_train.cutted_comment).toarray()
vectortemp=matrix[1,:]
def a(vetortemp):
    for idx in vectortemp.nozero()[0]:
        for i in range(int(vectortemp)):
            yield idx
# a(vectortemp)

In [17]:
vectortemp.nonzero()

(array([ 275,  695, 2041, 2507, 3781, 5312], dtype=int64),)

+ yield函数相当于迭代输出，是iteration+print的升级版，但是只能在函数内使用
+ https://www.ibm.com/developerworks/cn/opensource/os-cn-python-yield/
+ https://www.liaoxuefeng.com/discuss/001409195742008d822b26cf3de46aea14f2b7378a1ba91000/001446988326180213d739cf0ef433cab8b72d0bc1a1eaa000

In [18]:
type(vectortemp)

numpy.ndarray

**多项式分布**

In [59]:
np.random.multinomial(1,[0.3,0.4,0.6])

array([1, 0, 0])

这里的1指后面的那几个概率，不得超过1.

In [60]:
a=np.random.multinomial(1,[0.3,0.2,0.6])
print(a,a.argmax())

[0 1 0] 1


argmax返回的是最大数的索引(索引从0开始)

In [18]:
term_matrix.head()

,00,01,10,100,11,12,120,12331,127,13,...,鼎泰,鼎泰丰,鼎鼎有名,齐且,齐全,齐刚,齐名,齐后,齐餐,龟速
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
term_matrix.shape

(1500, 7305)

训练集一共1500行，7305列，也就是说有7305个特征，很棉线概念，前面有很多数字都被当成了特征，这是有问题的，所以使用停用词是有必要的

In [35]:
vect_stop=CountVectorizer(stop_words=frozenset(stopwords))

frozenset() 返回一个冻结的集合，冻结后集合不能再添加或删除任何元素。 也就是一个不可变集合

In [36]:
term_stop_metrix=pd.DataFrame(vect_stop.fit_transform(X_train.cutted_comment).toarray(),columns=vect_stop.get_feature_names())

注意，这里的vect_stop.fit_transform(X_train.cutted_comment)必须后面要跟着 toarray() 不然会报错，DF以不合适的方式被调用

In [37]:
term_stop_metrix.head()

,00,01,10,100,11,12,120,12331,127,13,...,鼎泰,鼎泰丰,鼎鼎有名,齐且,齐全,齐刚,齐名,齐后,齐餐,龟速
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
term_stop_metrix.shape

(1500, 7144)

+ 至此，特征数从7305减到7144，减少的161个特征，就是停用词表中的单词。
+ 但是，即便使用了停用词，还是会有很多漏网之鱼，比如这些数字，没有单位，没有上下文的数字都是没有意义的

#### 进一步减少特征（去除无意义的数字特征等）

在python中，可以通过设置 ^$ token_pattern ^$来实现这个操作，此处需要用到正则表达式的知识

+ 另一个问题在于，我们看到的这个矩阵，实际上是个非常稀疏的矩阵，其中大部分的取值都是0
+ 这也很正常，毕竟大部分评论语句当中都只有几十个词语而已。7K多特征，单个语句显然覆盖不过来
+ 但是，有些词汇作为特征，就值得注意了：
    + 首先是那些过于普遍的词汇。尽管使用了停用词表，但是难免有些词汇还是会几乎出现在每一句评论中
    + 什么是特征，特征就是可以把一个事物与其他事物区分开的属性
    + 同时，物极必反，那些过于特殊的词汇，其实也不应该保留。因为你了解了这个特征之后，对自己的模型而言，在处理新的语句的情感判断时，几乎都用不上

所以如下所示，一共多设置了三层特征词汇过滤

In [39]:
max_df=0.8 #单词出现的句子/1500(总的文档数)  比例超过0.8 太过平凡，删掉
min_df=3  # 出现次数低于3次（只有3个句子出现了这个词） 太过特殊 删掉

In [40]:
vect_full=CountVectorizer(max_df=max_df,
                         min_df=min_df,
                         token_pattern=u'(?u)\\b[^\\d\\W]\\w+\\b',
                         stop_words=frozenset(stopwords))

In [41]:
term_full_metrix=pd.DataFrame(vect_full.fit_transform(X_train.cutted_comment).toarray(),columns=vect_full.get_feature_names())

In [42]:
term_full_metrix.head()

,ipad,ok,ps,一下,一个个,一个半,一个多,一人,一份,一会,...,麻将,麻烦,麻辣,麻酱,麻麻,黄瓜,黄盖,黑椒,默默,齐全
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
term_full_metrix.shape

(1500, 1864)

现在特征数明显减少，
1. 什么都不做，共有7305个特征词
2. 只使用停用词表，减到7144，减少了161个
3. 限定词频，删除数字，减到1864，减少了几乎4k

特征多，不一定是好事，尤其是噪声大量混入时，会显著影响模型的性能

### 训练模型

数据已经进行了特征向量化，处理好之后就可以使用特征矩阵来训练模型了，使用的是朴素贝叶斯模型(Multinomial naive bayes)

In [44]:
from sklearn.naive_bayes import MultinomialNB

In [45]:
nb=MultinomialNB()

很明显，我们处理数据的流程是：
1. 特征向量化；
2. 朴素贝叶斯分类；

如果每次修改一个参数，或者换用测试集，就要重新运行一堆函数，这很麻烦，所以Sciki-learn提供了一个功能，叫管道（pipeline）,可以方便的解决这个问题

+ 管道（pipeline）可以帮助我们把这些顺序工作连接起来，隐藏其中的功能顺序关联，从外部一次调用，就能完成顺序定义的全部工作

In [46]:
from sklearn.pipeline import make_pipeline

In [47]:
pipe=make_pipeline(vect_full,nb)

In [48]:
pipe.steps

[('countvectorizer',
  CountVectorizer(analyzer='word', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=0.8, max_features=None, min_df=3,
          ngram_range=(1, 1), preprocessor=None,
          stop_words=frozenset({'倘或', '５：０  ', '进而', '对', '某', '［⑧］', 'ｂ］', '@', '依', '与其', '由于', '要不是', '以至', '［①⑥］', '而外', '不单', '该', '赶', '假如', '那个', '任', '啪达', '［②⑩］', '甚至', '呀', '＝″', '之类', '诸位', '可是', '虽', '.日 ', '唉', '那么', '其中', '有的是', '啥', 'ＬＩ', '得', '”，', '乃', '【', '似的', '前后', '那么些', '咚', '［②］', '并',... '嘎', '＿', '比方', '一', '也好', '？', '哪怕', '至', '他们', '∪φ∈', '有', '比', '［①③］', '［①ｆ］', '同', '［⑩］', '呕'}),
          strip_accents=None, token_pattern='(?u)\\b[^\\d\\W]\\w+\\b',
          tokenizer=None, vocabulary=None)),
 ('multinomialnb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]

可以看到，其包含了我们刚刚所做的工作

** 所以可以把管道当成一个整体模型来调用 **

很明显可以感觉得到，SVM运行起来比LDA快，毕竟后者要计算的量很大

In [49]:
from sklearn.model_selection import cross_val_score

In [50]:
cross_val_score(pipe,X_train.cutted_comment,Y_train,cv=5,scoring='accuracy').mean()

0.820687244673089

+ 上面这句代码是 使用交叉验证，计算模型分类准确率的平均值，
    + 这里的cv参数就是控制几倍交叉验证的，默认为3倍交叉验证，   

k 折交叉验证通过对 k 个不同分组训练的结果进行平均来减少方差，因此模型的性能对数据的划分就不那么敏感。

    第一步，不重复抽样将原始数据随机分为 k 份。
    第二步，每一次挑选其中 1 份作为测试集，剩余 k-1 份作为训练集用于模型训练。
    第三步，重复第二步 k 次，这样每个子集都有一次机会作为测试集，其余机会作为训练集。
    在每个训练集上训练后得到一个模型，用这个模型在相应的测试集上测试，计算并保存模型的评估指标，
    第四步，计算 k 组测试结果的平均值作为模型精度的估计，并作为当前 k 折交叉验证下模型的性能指标。
+ 可以发现，模型在训练集中的准确率为0.82068

能不能也像以前在SVM鸢尾花分类的时候一样，抽取重要的特征，舍弃不重要的。

现在有1864个特征，那就用1350个试试好了，哈哈哈哈,但是只使用特征抽取，我不知道怎么去赋值新的每列特征值的名字，，，，，，先放弃

回忆一下，一开始的数据选择是正负情感各占50%

如果我们建立一个一刀切的“笨模型”（dummy model）,即所有的评论，都当成正向（或负向）情感，准确率为50%，

**而目前模型的准确率超出30%，其实就是评论信息给模型带来的确定性**

In [64]:
pipe.fit(X_train.cutted_comment,Y_train)

Pipeline(memory=None,
     steps=[('countvectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.8, max_features=None, min_df=3,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=...e, vocabulary=None)), ('multinomialnb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

刚刚上面的只是交叉验证，这里我们用训练集把模型拟合出来

然后在测试集上，对情感分类标记进行预测

In [65]:
pipe.predict(X_test.cutted_comment)

array([0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,

测试集一共是500条数据

接下来看看预测结果和实际结果比较后的准确率

In [66]:
from sklearn import metrics

In [68]:
Y_pred=pipe.predict(X_test.cutted_comment)

In [69]:
metrics.accuracy_score(Y_pred,Y_test)

0.86

比在训练集上的准确率还高。。。

训练集上的准确率0.820687244673089

对于分类问题，只看准确率有些不全面，来看看混淆矩阵

In [70]:
metrics.confusion_matrix(Y_test,Y_pred)

array([[194,  43],
       [ 27, 236]], dtype=int64)

混淆矩阵中的4个数字，分别代表：

    TP: 本来是正向，预测也是正向的；
    FP: 本来是负向，预测却是正向的；
    FN: 本来是正向，预测却是负向的；
    TN: 本来是负向，预测也是负向的。

![avatar](https://upload-images.jianshu.io/upload_images/64542-0c881f7d9c516532..png?imageMogr2/auto-orient/strip%7CimageView2/2/w/699)

使用dummy model(笨模型)对比是没有价值的，使用SnowNLP来试试

In [72]:
from snownlp import  SnowNLP

In [73]:
def get_sentiment(text):
    return SnowNLP(text).sentiments

In [75]:
Y_pred_snownlp=X_test.comment.apply(get_sentiment)

这里有个小问题。 SnowNLP 生成的结果，不是0和1，而是0到1之间的小数。

所以我们需要做一步转换，把0.5以上的结果当作正向，其余当作负向。

In [76]:
Y_pred_snownlp_normalized=Y_pred_snownlp.apply(lambda x:1 if x>0.5 else 0)

In [77]:
Y_pred_snownlp_normalized[:5]

674     0
1699    1
1282    0
1315    0
1210    0
Name: comment, dtype: int64

查看下情感值转换后的结果，满足要求

In [78]:
metrics.accuracy_score(Y_test,Y_pred_snownlp_normalized)

0.77

所以这就算一个对照试验了，SVM模型跑出来的要比SnowNLP训练使用的模型高了

0.86-0.77=0.09

In [79]:
metrics.confusion_matrix(Y_test,Y_pred_snownlp_normalized)

array([[189,  48],
       [ 67, 196]], dtype=int64)

看看混淆矩阵，进行对比，很明显，在TP和TN两项上，SVM模型判断正确数量，都要超过SnowNLP